<h1>CLASIFICACIÓN MULTICLASE<h1>


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# utilizado para la manipulación de directorios y rutas
import os
#importamos pandas para el manejo del dataset, y separarlos dentro de una matriz
import pandas as pd
# Cálculo científico y vectorial para python
import numpy as np
from sklearn.metrics import confusion_matrix
# Libreria para graficos
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
# Modulo de optimizacion en scipy
from scipy import optimize
# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [3]:
input_layer_size  = 22
# tengo 3 clases
num_labels = 3
#se pone 7 a pesar de haber 6 clases con esto el 0 no se toma en cuenta (como es uno contra todos hay que realizar este artificio para considerar todas las clases u otra opcion es convertir el 6 en 0)(el argmmax nos da 0 1 2 y para tomar en cuenta las 3 clases aumentamos 1 mas y quitamos el 0)
data = np.loadtxt('/content/drive/MyDrive/SIS420 Inteligencia Artificial/machine learning/datasets/diabetes_012_health_indicators_BRFSS2015.csv', delimiter=',', skiprows=1)

# Separar las características (X) de las etiquetas (y)
X = data[:, 1:]  # Todas las filas, todas las columnas excepto la primera
y = data[:, 0]   # Todas las filas, primera columna

# Número de ejemplos
m = y.size



In [4]:
print(X[0, :])
print(y)

[ 1.  1.  1. 40.  1.  0.  0.  0.  0.  1.  0.  1.  0.  5. 18. 15.  1.  0.
  9.  4.  3.]
[0. 0. 0. ... 0. 0. 2.]


In [ ]:
# Calcular el conteo de cada clase
clases, conteo = np.unique(y, return_counts=True)

# Imprimir el conteo de cada clase
for clase, count in zip(clases, conteo):
    print(f"Clase {int(clase)}: {count} ejemplos")

Clase 0: 213703 ejemplos
Clase 1: 4631 ejemplos
Clase 2: 35346 ejemplos


<h2> NORMALIZACION <h2>

In [ ]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [ ]:
X_norm, mu, sigma = featureNormalize(X)

In [ ]:
m, n = X.shape
X = X_norm

<h2> FUNCION SIGMOIDE <h2>

In [ ]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

<h2> COSTO POR EL GRADIENTE <h2>

In [ ]:
def lrCostFunction(theta, X, y, lambda_):
    m = y.size
    if y.dtype == bool:
        y = y.astype(int)

    #     alpha = 0.003
    #     theta = theta.copy()
    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

    #aplicando regularizacion
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    #     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = (1 / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
    #return J, theta

<h2> CLASIFICACIÓN ONE VS ALL <h2>

In [ ]:
def oneVsAll(X, y, num_labels, lambda_):
    m, n = X.shape
    all_theta = np.zeros((num_labels, n + 1))

    # Agregar unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [ ]:
lambda_ = 0.1
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)

(3, 22)


In [ ]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0]
    num_labels = all_theta.shape[0]
    p = np.zeros(m)

    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis=1)
    return p


In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Calcula el número de datos para el conjunto de entrenamiento (80%)
#total_samples = len(X)
#num_train = int(0.8 * total_samples)

# Dividir los datos en conjuntos de entrenamiento y prueba
#X_train, X_test = X[:num_train], X[num_train:]
#y_train, y_test = y[:num_train], y[num_train:]
# Entrenar el modelo utilizando todos los datos
#lambda_ = 0.01
#all_theta_train = oneVsAll(X_train, y_train, num_labels, lambda_)



# Entrenar el modelo con los datos de entrenamiento
all_theta_train = oneVsAll(X_train, y_train, num_labels, lambda_)

(253680, 21)
Precision del conjuto de entrenamiento: 84.64%


In [ ]:
# Calcular el conteo de ejemplos por clase
clases, conteo = np.unique(y_train, return_counts=True)

# Imprimir el conteo de ejemplos por clase
for clase, count in zip(clases, conteo):
    print(f"Clase {int(clase)}: {count} ejemplos")

# Verificar si hay clases desbalanceadas
if np.any(conteo < 0.5 * np.mean(conteo)):
    print("Se detecta un desbalance significativo en las clases.")

    # Aplicar técnicas de balanceo de clases aquí, por ejemplo:
    # 1. Oversampling: Aumentar el número de instancias de las clases minoritarias
    # 2. Undersampling: Reducir el número de instancias de las clases mayoritarias
    # 3. SMOTE: Generar nuevas instancias sintéticas de las clases minoritarias
    # 4. Otros métodos de balanceo de clases


Clase 0: 170908 ejemplos
Clase 1: 3687 ejemplos
Clase 2: 28349 ejemplos
Se detecta un desbalance significativo en las clases.


<h2> PREDICCIÓN ONE VS ALL <h2>

In [ ]:
# Predecir las clases utilizando el conjunto de entrenamiento
pred_train = predictOneVsAll(all_theta_train, X_train)

# Calcular la precisión del conjunto de entrenamiento
accuracy_train = np.mean(pred_train == y_train) * 100
print('Precisión del conjunto de entrenamiento: {:.2f}%'.format(accuracy_train))

# Predecir las clases utilizando el conjunto de prueba
pred_test = predictOneVsAll(all_theta, X_test)

# Calcular la precisión del conjunto de prueba
accuracy_test = np.mean(pred_test == y_test) * 100
print('Precisión del conjunto de prueba: {:.2f}%'.format(accuracy_test))

Precisión del conjunto de entrenamiento: 84.60%
Precisión del conjunto de prueba: 84.84%


In [ ]:
conf_matrix = confusion_matrix(y_test, pred_test)

# Mostrar la matriz de confusión
print("Matriz de Confusión:")
print(conf_matrix)

Matriz de Confusión:
[[41770     0  1025]
 [  872     0    72]
 [ 5725     0  1272]]


In [ ]:
# Ejemplo de predicción para nuevos datos
X_prueba = np.array([[1,1,1,40,1,0,0,0,0,1,0,1,0,5,18,15,1,0,9,4,3],
                     [1,1,0,27,0,0,0,1,1,1,0,1,0,2,0,0,0,0,5,3,6],
                     [1,1,1,27,0,0,0,1,1,1,0,1,1,4,30,30,1,0,10,4,3],
                     [0,1,1,31,1,0,0,1,1,1,0,1,0,3,0,1,0,0,12,2,2],
                     [0,0,1,23,0,0,0,1,0,1,0,1,0,2,2,0,0,0,10,4,5],
                     [0,1,1,2,0,0,0,1,1,1,0,1,0,2,0,0,0,1,0,6,8]])

p_prueba = predictOneVsAll(all_theta, X_prueba)
print("Predicciones para nuevos datos:", p_prueba)

Predicciones para nuevos datos: [2 2 2 2 2 0]


In [ ]:
print(all_theta)

[[ 2.21984415e+00 -3.52458334e-01 -2.93743768e-01 -2.25450465e-01
  -4.09354808e-01  6.83095088e-03 -2.12672583e-02 -5.79557520e-02
   2.01992497e-02  2.24179201e-02  1.43973124e-02  1.54955240e-01
  -1.19456719e-02 -2.08692672e-02 -5.49027489e-01  1.50811960e-02
   6.15802467e-02 -4.04829687e-02 -1.19025409e-01 -3.86980078e-01
   3.82369975e-02  1.15803764e-01]
 [-4.22667240e+00  1.32835273e-01  2.27440444e-01  1.36994891e-01
   2.03477144e-01 -5.14772970e-03 -3.61199941e-02 -4.11507602e-02
   1.47319811e-05 -8.28624439e-03 -1.69775680e-02 -1.25654256e-02
  -2.55370802e-02  9.42155735e-02  2.09578098e-01  5.73363223e-02
  -4.37354291e-02 -3.16647350e-02  2.66608703e-02  3.08533562e-01
  -6.31641777e-02 -1.09609932e-01]
 [-2.41685018e+00  3.74928271e-01  2.85960535e-01  2.35702839e-01
   4.02241739e-01 -5.13833239e-03  2.64642691e-02  6.44251902e-02
  -2.22320196e-02 -2.40609580e-02 -1.29820551e-02 -1.77117523e-01
   1.78435201e-02  4.95742468e-03  5.72596795e-01 -2.69254023e-02
  -6.4